In [3]:
from sde_2d_v01 import Sde_2d, Heston
from contract_v01 import VanillaOption

In [4]:
#initiate option and heston instances
heston1 = Heston(init_state=[100., .04], r=.05, 
                 kappa=1.2, theta=.04, xi=.3, rho=.5)

In [9]:
'''=========
geometric asian option class
=============='''
class geometric_asian_option:
    def __init__(self,
                maturity = 2/12,
                strike = 101,
                num_time_step = 8
                ):
        self.maturity = maturity
        self.strike = strike
        self.num_time_step = num_time_step
        

In [12]:
'''============
initiate option class list
=============='''
strike_list = [97,99,101,103]
maturity_list = [2/12, 5/12]
option_list = []
for t in maturity_list:
    for k in strike_list:
        option1 = geometric_asian_option(maturity=t, strike=k, num_time_step=8)
        option_list.append(option1)